In [38]:
from numpy.random import seed
import cv2
import os
import numpy as np
import skimage
from skimage.transform import resize
import csv
import itertools
from tqdm import tqdm
from glob import glob
import random
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from sklearn.model_selection import train_test_split
import keras

from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Conv2D, Dense, Dropout, Flatten
from keras.layers import Flatten, Dense
from keras.models import Sequential

In [29]:
def load_images(directory):
    images = []
    labels = []
    uniq_labels = sorted(os.listdir(directory))
    for idx, label in tqdm(enumerate(uniq_labels)):
        for file in os.listdir(directory + "/" + label):
            filepath = directory + "/" + label + "/" + file
            image = cv2.resize(cv2.imread(filepath), (64, 64))
            images.append(image)
            labels.append(idx)
        break
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

In [20]:
train_dir = "./archive/asl_alphabet_train/asl_alphabet_train/"
x, y = load_images(train_dir)

0it [00:02, ?it/s]


In [32]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y)
x_train.shape, y_train.shape

((2400, 64, 64, 3), (2400,))

In [34]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)
x_train = x_train.astype(np.float32) / 255.
x_test = x_test.astype(np.float32) / 255.

In [36]:
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=5, padding='same', activation='relu', input_shape=(64, 64, 3)))
model.add(Conv2D(filters=64, kernel_size=5, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Dropout(0.5))

model.add(Conv2D(filters=128, kernel_size=5, padding='same', activation='relu'))
model.add(Conv2D(filters=128, kernel_size=5, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Dropout(0.5))

model.add(Conv2D(filters=256, kernel_size=5, padding='same', activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(29, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])
model.summary()

W1008 22:24:38.656214 4487886272 module_wrapper.py:139] From /Users/brennangebotys/miniconda2/envs/meta/lib/python3.6/site-packages/keras/optimizers.py:711: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1008 22:24:38.712070 4487886272 deprecation.py:506] From /Users/brennangebotys/miniconda2/envs/meta/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:2885: calling reduce_sum_v1 (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
W1008 22:24:38.737643 4487886272 module_wrapper.py:139] From /Users/brennangebotys/miniconda2/envs/meta/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:2889: The name tf.log is deprecated. Please use tf.math.log instead.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d

In [37]:
epochs = 100 
batch_size = 64

results = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)

In [ ]:
test_results = model.evaluate(x=x_test, y=y_test)
print('Accuracy for test images:', round(test_results[1]*100, 3), '%')